In [1]:
import collections
from collections import OrderedDict
import pandas as pd
import re
import string
from nltk import word_tokenize, wordpunct_tokenize
import itertools

In [2]:
#load morphed text
import pandas as pd
import csv
a = pd.read_csv('lehtisalo_8_morf.txt', sep='\t', quoting=csv.QUOTE_NONE,
                names=['Cyrillic token', 'Munkácsi token', 'Wichmann token', 'Steinitz token', 'SzOCh token',
                       'RME token', 'Hajdú token', 'Mus token', 'IPA token', 'segmented token',
                       'lemma', 'Hungarian gloss', 'English gloss', 'POS tag', 'RUS/-'])

In [3]:
a.head()

,Cyrillic token,Munkácsi token,Wichmann token,Steinitz token,SzOCh token,RME token,Hajdú token,Mus token,IPA token,segmented token,lemma,Hungarian gloss,English gloss,POS tag,RUS/-
0,тадебе,-,-,-,-,-,tādʼebe,tadʼebe,tadʲebe,-,тадебя,-,shaman,N,-
1,выʼ,-,-,-,-,-,wīʔ,wiʔ,wiːʔ,-,выʼ,-,tundra,N,-
2,.,-,-,-,-,-,.,.,.,-,.,-,.,.,-
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ңавнанта,-,-,-,-,-,ŋāwnanta,ŋawnanta,ŋawnăntă,-,ңавна,-,previously:Gen.PxSg3,Adv.Temp,-


In [4]:
#drop all rows with NaN
a = a.dropna(how='all').reset_index() 
a.head()

,index,Cyrillic token,Munkácsi token,Wichmann token,Steinitz token,SzOCh token,RME token,Hajdú token,Mus token,IPA token,segmented token,lemma,Hungarian gloss,English gloss,POS tag,RUS/-
0,0,тадебе,-,-,-,-,-,tādʼebe,tadʼebe,tadʲebe,-,тадебя,-,shaman,N,-
1,1,выʼ,-,-,-,-,-,wīʔ,wiʔ,wiːʔ,-,выʼ,-,tundra,N,-
2,2,.,-,-,-,-,-,.,.,.,-,.,-,.,.,-
3,4,ңавнанта,-,-,-,-,-,ŋāwnanta,ŋawnanta,ŋawnăntă,-,ңавна,-,previously:Gen.PxSg3,Adv.Temp,-
4,5,хунананта,-,-,-,-,-,xunanānta,xunananta,xunănantă,-,хуна,-,where:Indet.Loc.PxSg3,Pron,-


In [5]:
#load translation file
'''
with open('lehtisalo3_eng.txt') as f2:
    lines = f2.readlines()
for i in range(len(lines)):
    lines[i] = lines[i][2:].strip()
    
    
def translation_to_sentences(original_lines=lines):
#doing some magic to align our data
    new_lines = []
    for line in original_lines:
        line = line.replace('."','&1@')
        line = line.replace('?"','&2@')
        line = line.replace('!"','&3@')
        split_line = line.replace('.','.@').replace('!','!@').replace('?', '?@').split('@')
        for j in split_line:
            if j != '':
                j = j.replace('&1', '."').replace('&2', '?"').replace('&3', '!"')
                new_lines.append(j)
                
    return new_lines


translation_lines = translation_to_sentences()      
'''

'\nwith open(\'lehtisalo3_eng.txt\') as f2:\n    lines = f2.readlines()\nfor i in range(len(lines)):\n    lines[i] = lines[i][2:].strip()\n    \n    \ndef translation_to_sentences(original_lines=lines):\n#doing some magic to align our data\n    new_lines = []\n    for line in original_lines:\n        line = line.replace(\'."\',\'&1@\')\n        line = line.replace(\'?"\',\'&2@\')\n        line = line.replace(\'!"\',\'&3@\')\n        split_line = line.replace(\'.\',\'.@\').replace(\'!\',\'!@\').replace(\'?\', \'?@\').split(\'@\')\n        for j in split_line:\n            if j != \'\':\n                j = j.replace(\'&1\', \'."\').replace(\'&2\', \'?"\').replace(\'&3\', \'!"\')\n                new_lines.append(j)\n                \n    return new_lines\n\n\ntranslation_lines = translation_to_sentences()      \n'

In [6]:
def make_sentences_from_data(data=a):
                             #translation=translation_lines):
    all_sentences = []
    sentence = ''
    glosses = ''
    morphemes = ''
    skip = False
    sentence_count = 0

    for i in range(len(a)):
        if skip:
            skip = False
            continue
        if a['Cyrillic token'][i] != '.' and a['Cyrillic token'][i] != '?' and a['Cyrillic token'][i] != '!':
            glosses += a['English gloss'][i] + ' '
            morphemes += a['lemma'][i] + ' '
            if a['Cyrillic token'][i] not in string.punctuation:
                sentence += a['Cyrillic token'][i] + ' '
            elif a['Cyrillic token'][i] == '"':
                sentence += a['Cyrillic token'][i]
            else:
                sentence = sentence.strip() + a['Cyrillic token'][i] + ' '
        else:
            glosses += a['English gloss'][i] + ' '
            morphemes += a['lemma'][i]
            sentence = sentence.strip()
            sentence += a['Cyrillic token'][i]
            skip = False
            if i < len(a) - 1:
                if a['Cyrillic token'][i+1] =='"':
                    sentence += a['Cyrillic token'][i+1]
                    skip = True
            all_sentences.append(OrderedDict({
                'sentence': sentence,
                'morphemes': morphemes,
                'glosses': glosses
                #'translation': translation[sentence_count]
            }))
            sentence = ''
            glosses = ''
            morphemes = ''
            sentence_count += 1
            

    return all_sentences

In [7]:
df = pd.DataFrame(make_sentences_from_data())
df.head()

,sentence,morphemes,glosses
0,тадебе выʼ.,тадебя выʼ .,shaman tundra .
1,ңавнанта хунананта я мидахана ңобʼ весоку выри...,ңавна хуна я мись ңобʼ вэсако вырире серодета ...,previously:Gen.PxSg3 where:Indet.Loc.PxSg3 lan...
2,пада сиде неде ңэвы.,пыда сидя не ңэсь .,he:Sg3 two wife:PxSg3 be:Narr.ScSg3 .
3,"ңобʼ тане хуна вырире ма: ""сидяв пухув, тебтаʼ...","ңобʼ таняʼ хуна вырире манзь : "" сидя пуху , т...",one to there:Spat.Dat where:Indet Virire tell:...
4,"тикы есе инявна манеʼ нумтʼ танаңкунаʼ.""",тикы еся иня маняʼ нумʼ танась .,this iron rope:Pros we:Pl1 sky:Dat climb up:Fu...


In [8]:
def string_into_tokens(s):
    ll = [[word_tokenize(w), ' '] for w in s.split()]
    s_list = list(itertools.chain(*list(itertools.chain(*ll))))
    return s_list

In [9]:
def make_sentence_words_arr(all_sentences, i):
    wf_arr = []
    tokens = string_into_tokens(all_sentences['sentence'][i])
    morphemes = all_sentences['morphemes'][i].split()
    glosses = all_sentences['glosses'][i].split()
    j = 0
    off_start = 0
    sentence_index = 0
    for k in range(len(tokens)):
        if tokens[k] == ' ':
            off_start += len(tokens[k])
            continue
        elif tokens[k] in string.punctuation:
            wf_arr.append(collections.OrderedDict({
                'wf': tokens[k],
                'wtype': 'punct',
                'off_start': off_start,
                'off_end': off_start + len(tokens[k]),
                'next_word': sentence_index+1,
                'sentence_index': sentence_index
                #'ana': False
            }))
            sentence_index += 1
        else:
            wf_arr.append(collections.OrderedDict({
                'wf': tokens[k],
                'wtype': 'word',
                'off_start': off_start,
                'off_end': off_start + len(tokens[k]),
                'next_word': sentence_index+1,
                'sentence_index': sentence_index,
                'ana': [{'glosses': glosses[j], 'morphemes': morphemes[j]}]
                #"ana": [{"lex": "ojorsp", "gr.pos": "V", "gr.pers": "2", "gr.tense": "t1"}]
            }))
            j += 1
            sentence_index += 1
        off_start += len(tokens[k])
    return wf_arr



In [10]:
pd.DataFrame(make_sentence_words_arr(df, 2))

,wf,wtype,off_start,off_end,next_word,sentence_index,ana
0,пада,word,0,4,1,0,"[{'glosses': 'he:Sg3', 'morphemes': 'пыда'}]"
1,сиде,word,5,9,2,1,"[{'glosses': 'two', 'morphemes': 'сидя'}]"
2,неде,word,10,14,3,2,"[{'glosses': 'wife:PxSg3', 'morphemes': 'не'}]"
3,ңэвы,word,15,19,4,3,"[{'glosses': 'be:Narr.ScSg3', 'morphemes': 'ңэ..."
4,.,punct,19,20,5,4,NaN


In [11]:
all_sentences = []

for i in range(len(df)):
    all_sentences.append(collections.OrderedDict({
            'text': df['sentence'][i],
            'words': make_sentence_words_arr(df, i),
            'lang': 0
        }))
    
#print(all_sentences)

In [12]:
pd.DataFrame(all_sentences).head()

,text,words,lang
0,тадебе выʼ.,"[{'wf': 'тадебе', 'wtype': 'word', 'off_start'...",0
1,ңавнанта хунананта я мидахана ңобʼ весоку выри...,"[{'wf': 'ңавнанта', 'wtype': 'word', 'off_star...",0
2,пада сиде неде ңэвы.,"[{'wf': 'пада', 'wtype': 'word', 'off_start': ...",0
3,"ңобʼ тане хуна вырире ма: ""сидяв пухув, тебтаʼ...","[{'wf': 'ңобʼ', 'wtype': 'word', 'off_start': ...",0
4,"тикы есе инявна манеʼ нумтʼ танаңкунаʼ.""","[{'wf': 'тикы', 'wtype': 'word', 'off_start': ...",0


In [13]:
one_text = {
    "sentences": all_sentences,
    "meta": {"author": "Lehtisalo, Toivo", "title": "Juraksamojedische Volksdichtung. Suomalais-Ugrilainen Seura, Helsinki", "year": 1947, "id": 2}
}

In [14]:
import json
with open('data_old8.json', 'w', encoding='utf-8') as f:
    json.dump(one_text, f, ensure_ascii=False, indent=4)